In [1]:
import os
import sys
import traci
import sumolib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from xml.dom import minidom
from sumolib import checkBinary
import math
from scipy.stats import norm
import ray
import pickle

In [2]:
# check env
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    raise EnvironmentError("Please declare environment variable 'SUMO_HOME'")

In [3]:
model_folder="model_params/"

PT_param_data={0: pd.read_csv(model_folder+"merged_PT_S.csv"),
                1: pd.read_csv(model_folder+"merged_PT_A.csv"),
                   2: pd.read_csv(model_folder+"merged_PT_L.csv")
                  }
# IDM by vehicle type
IDM_param_data={0: pd.read_csv(model_folder+"merged_IDM_S.csv"),
                   1: pd.read_csv(model_folder+"merged_IDM_A.csv"),
                   2: pd.read_csv(model_folder+"merged_IDM_L.csv")}

In [4]:
max(IDM_param_data[0]["v0"].values)

35.12486718879205

In [5]:
    i=0
    print("T", np.mean(IDM_param_data[i]["T"].values), np.mean(IDM_param_data[i+1]["T"].values), np.mean(IDM_param_data[i+2]["T"].values) )
    print("a", np.mean(IDM_param_data[i]["a"].values), np.mean(IDM_param_data[i+1]["a"].values), np.mean(IDM_param_data[i+2]["a"].values) )
    print("b", np.mean(IDM_param_data[i]["b"].values), np.mean(IDM_param_data[i+1]["b"].values), np.mean(IDM_param_data[i+2]["b"].values) )
    print("v0", np.mean(IDM_param_data[i]["v0"].values), np.mean(IDM_param_data[i+1]["v0"].values), np.mean(IDM_param_data[i+2]["v0"].values) )
    print("so", np.mean(IDM_param_data[i]["so"].values), np.mean(IDM_param_data[i+1]["so"].values), np.mean(IDM_param_data[i+2]["so"].values) )
    print("delta", np.mean(IDM_param_data[i]["delta"].values), np.mean(IDM_param_data[i+1]["delta"].values), np.mean(IDM_param_data[i+2]["delta"].values) )

T 1.4625595911390286 1.5290589527536702 1.8716851688470852
a 1.447996117609171 1.4972376240940442 1.4354862369471078
b 1.6527573870057988 1.871281964263802 1.3034966589917416
v0 27.98608288804145 27.977432375855066 24.803077579499412
so 3.623163003417833 3.639212988199135 3.5378427259878995
delta 4.18627224094722 4.0592790889160115 4.020322736768535


In [ ]:
def initialize():
    od_demand={} # the demands are stored as dictionaries of dictionary
    if site=="I90_94":
        in_names={1:"main_in", 2:"on1"}
        out_names={1:"off1",2:"off2", 3:"off3", 4:"off4", 5:"main_I90_out", 6:"main_I94_out" }
        downstream_outs={1:{1,2,3,4,5,6}, 2:{2,3,4,5,6}} # make sure that the exist is downstream of the entrance
        # define edges
        in_edges={1:"124471132", 2:"24521704#3.24"}
        out_edges={1:"24306156",2:"24521700#0", 3:"27878350", 4:"27862401", 5:"380664935", 6:"907236679" }
        # user defined demand
        # To do. Not sure how we define when MLCs are made
        in_MLCs={"124471132":{}, "24521704#3.24":{"121980283":1}}
        out_MLCs={"24306156":{}, "24521700#0":{}, "27878350":{}, "27862401":{}, "380664935":{}, "907236679":{}}
        with open('next_lanes_I90_94.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)
        
    elif site=="I294":
        in_names={1:"main_in", 2:"on1", 3:"on2", 4:"on3"}
        out_names={1:"off1", 2:"off2", 3:"main_out"}
        in_edges={1:"992387151#1", 2:"38878546.568", 3:"31144126.151", 4:"24067152.262"}
        out_edges={1:"24067154", 2:"24067155", 3:"377623249#1-AddedOnRampEdge"}
        downstream_outs={1:{1,2,3}, 2:{1,2,3}, 3:{3}, 4:{3} }
        
        # On what edges would MLCs and to what direction would they be executed.
        # To do
        in_MLCs={"61857431#1":{}, "61857388":{"121980283":1}, "24067978":{}, "38878546.568":{}, "31144126.151":{}, "24067152.262":{}}
        out_MLCs={"33726629":{},"24067154":{}, "24067155":{}, "377623249#1":{}}
        #print("reading_next_lane")
        with open('next_lanes_I294.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)
    
    elif site=="I395":
        in_names={1: "main_in", 2:"on1"}
        out_names={1: "off1", 2:"main_out"}
        in_edges={1:"6063187-AddedOffRampEdge", 2:"297204299"}
        out_edges={1:"397355152", 2:"121980283.9"}
        downstream_outs={1:{1,2}, 2:{2} }
        # user defined demand
        in_MLCs={"6063187-AddedOffRampEdge":{}, "297204299":{"121980283":-1}} # for each origin what 
        out_MLCs={"397355152":{"6063187-AddedOffRampEdge":-1}, "121980283.9":{}}
        #first_exit="397355152" # the name of the first exit
        #first_exit_start_lane="2" # vehicles who take the first exit are generated from this lane
        with open('next_lanes_I395.pickle', 'rb') as handle:
            next_lanes_dict = pickle.load(handle)
        
    
    # TOY CASE: IF MAIN TO MAIN then 1000, if one of them is main and the other one not, then 100, if both not main then 20
    # change the code below
    for in_idx in in_names:
        od_demand[in_idx]={}
        in_name=in_names[in_idx]
        for out_idx in out_names:
            out_name=out_names[out_idx]
            if not out_idx in downstream_outs[in_idx]:
                od_demand[in_idx][out_idx]=0 #if the offramp is upstream of the on ramp
            elif ("main" in in_name) and ("main" in out_name):
                od_demand[in_idx][out_idx]=flow_main
            elif (("main" in in_name) and ( not "main" in out_name)):
                od_demand[in_idx][out_idx]=flow_main_to_ramp
            elif ((not "main" in in_name) and ("main" in out_name)):
                od_demand[in_idx][out_idx]=flow_ramp_to_main
            else:
                od_demand[in_idx][out_idx]=flow_ramp_to_ramp
            print("From " + in_name+" to "+out_name+": ", od_demand[in_idx][out_idx], "veh/hr" )


    all_edges=[]
    # Load the XML file
    parsed_data = ET.parse('network_structure/'+site+'.net.xml')
    root = parsed_data.getroot()
    
    print(f"Root tag: {root.tag}, Root attributes: {root.attrib}")
    
    # Iterate through elements to check structure
    for edge in root.findall(".//edge"):
        all_edges.append(edge.attrib['id'])
        #print(f"Edge ID: {edge.attrib['id']} - Function: {edge.attrib.get('function')}")
        #for lane in edge.findall("lane"):
            #print(f"\tLane ID: {lane.attrib['id']}, Speed: {lane.attrib['speed']}")

    return od_demand, all_edges, in_edges, out_edges, 0, 0, next_lanes_dict

def generate_vehicles(trial=None):
    all_generation_times={}
    origins=[]
    destinations=[]
    gen_times=[]
    
    for in_idx in od_demand:
        in_edge=in_edges[in_idx]
        all_generation_times[in_edge]={}
        for out_idx in od_demand[in_idx]:
            out_edge=out_edges[out_idx]
            demand_rate=od_demand[in_idx][out_idx]
            if demand_rate>0:
                generation_times=np.cumsum(np.maximum(np.random.exponential(3600/demand_rate, 100000), min_hw))
                generation_times=generation_times[(generation_times>=min_t) & (generation_times<=max_t)]
                
                #all_generation_times[in_edge][out_edge]=generation_times
                for ll in range(len(generation_times)):
                    origins.append(in_edge)
                    destinations.append(out_edge)
                    gen_times.append(generation_times[ll])
    
    
    origins=np.array(origins)
    destinations=np.array(destinations)
    gen_times=np.array(gen_times)
    
    
    sort_indices=np.argsort(gen_times)
    gen_times=gen_times[sort_indices]
    origins=origins[sort_indices]
    destinations=destinations[sort_indices]
    
    vehicle_ids=np.array([idx+1 for idx in range(len(gen_times))])

    PTs_by_id={}
    IDMs_by_id={}
    types_by_id={}
    lens_by_id={}
    
    trips_data=[]
    
    for i in range(len(vehicle_ids)):#trip in trips_data:
        id=str(vehicle_ids[i])
        
        veh_type=np.random.choice([0,1,2], p=[sv_rate, av_rate, lv_rate])
        types_by_id[id]=veh_type
        lens_by_id[id]=(veh_type==2)*12+(veh_type!=2)*4

        
    
        PT_cf_data=PT_param_data[int(veh_type)]
        sample_PT=PT_cf_data.sample()[['Tmax', 'Alpha', 'Beta', 'Wc', 'Gamma1','Gamma2', 'Wm']].values[0]
        PTs_by_id[id]=sample_PT
    
        IDM_cf_data=IDM_param_data[int(veh_type)]
        sample_IDM=IDM_cf_data.sample()[['T', 'a', 'b', 'v0', 'so', "delta"]].values[0]
        IDMs_by_id[id]=sample_IDM

        veh={"id": id, "depart": str(round(gen_times[i],1)), "from": origins[i] , "to": destinations[i], "type": veh_type, "departSpeed":sample_IDM[3] }
        trips_data.append(veh)
    

    root = ET.Element("trips")

    #veh_types=ET.SubElement(root, "vType")

    # Add trips to the root
    for trip in trips_data:
        trip_element = ET.SubElement(root, "trip")
        trip_element.set("id", str(trip["id"]))
        trip_element.set("depart", trip["depart"])
        trip_element.set("from", trip["from"])
        trip_element.set("to", trip["to"])
        #if trip["to"]!=first_exit:
        trip_element.set("departLane", "random") # random departure lane
        #else:
            #trip_element.set("departLane", first_exit_start_lane) 
        trip_element.set("departSpeed", str(trip["departSpeed"]))
        #if veh["type"]==0 or veh["type"]==1:
            #trip_element.set("")
        
        
    
    # Create an ElementTree object
    tree = ET.ElementTree(root)
    # Convert to a string and pretty print
    xml_str = minidom.parseString(ET.tostring(root, 'utf-8')).toprettyxml(indent="  ")
    
    # Save the XML to a file
    output_file = "configs/"+ site +"/"+site+"_trial"+str(trial)+".trips.xml"
    with open(output_file, "w") as file:
        file.write(xml_str)
    
    print(f"Trips saved to {output_file}")

    return PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data
    


In [ ]:
def PT(veh_id, leader_id):#vehicle, accl_max, v_desired, params, Tcorr, RT, prng):
    
    #PTs_by_id 
    ####### i, t, and RT are usedless right? Also prng,also Tcorr
    ####### What is So_D ????
    ####### Where do I get v_desired?
    ####### Use accl_max from the paper?
    Tmax, Alpha, Beta, Wc, Gamma1,Gamma2, Wm = PTs_by_id[veh_id] # read the PT parameters for the current vehicle
    # delta_v is speed difference
    # gap is the leader location minus the current location minus leader length
    # speed is the current speed of the central vehicle
    
    # central vehicle
    x_a = traci.vehicle.getLanePosition(veh_id)
    position=traci.vehicle.getPosition(veh_id)
    v_a=traci.vehicle.getSpeed(veh_id)
    road= traci.vehicle.getRoadID(veh_id)
    lane= traci.vehicle.getLaneIndex(veh_id)

    if leader_id!=None:
        leader_len=lens_by_id[leader_id]
        v_lead=traci.vehicle.getSpeed(leader_id)
        x_lead=traci.vehicle.getLanePosition(leader_id)
    else: # virtual vehicle needed, to be verified
        leader_len=0
        v_lead=v_a+0.01
        x_lead=10000000# make it huge

    veh_gap=x_lead-x_a-leader_len # gap between leader and the current vehicle
    veh_delta_v= v_a-v_lead # speed difference  

    So_D = 3 #default value by Talebpour
    if (veh_gap - So_D) > 0.1:
        Seff = veh_gap - So_D
    else:
        Seff = 0.1 #default value by Talebpour

    if veh_delta_v > (Seff / Tmax):
        Tau = Seff / veh_delta_v
    else:
        Tau = Tmax

    if veh_delta_v == 0:
        veh_delta_v = 0.0000001 #default value by Talebpour
    if Alpha == 0:
        Alpha = 0.0000001 #default value by Talebpour
    Zprime = Tau / (2.0 * Alpha * v_a)
    Zdoubleprime = 0.0

    #if Wc * Zprime >= 1:
    if Wc * Zprime > 0:
        if (2.0 * math.log(Wc * Zprime)) >= 0:
            a0 = 1
            #Zstar = (-1 * math.sqrt(2.0 * math.log(Wc * Zprime))) / (math.sqrt(2.0 * math.pi)) #default by Talebpour
            Zstar = -math.sqrt(2.0 * math.log((a0 * Wc * Tau) / (2.0 * math.sqrt(2.0 * math.pi) * Alpha * v_a))) #changed to be consistent with paper
            if np.abs(Zstar) > 0.05:
                Zstar = 0.05 #added threshold to reduce fluctuations
    else:
        Zstar = 0.0
    Astar = (2.0 / Tau) * ((Seff / Tau) - veh_delta_v + (Alpha * v_a * Zstar))
    for NewtonCounter in range(3):
        X = Astar
        if X >= 0:
            if X == 0:
                X = 0.0000001 #default value by Talebpour
            Uptprime = Gamma1 * math.pow(X, Gamma1 - 1)
            Uptdoubleprime = Gamma1 * (Gamma1 - 1) * math.pow(X, Gamma1 - 2)
        else:
            Uptprime = Wm * Gamma2 * pow(-X, Gamma2 - 1)
            Uptdoubleprime = -Wm * Gamma2 * (Gamma2 - 1) * pow(-X, Gamma2 - 2)

        Z = (veh_delta_v + (0.5 * Astar * Tau) - (Seff / Tau)) / (Alpha * veh_delta_v)
        fn = norm.cdf(Z)

        F = Uptprime - Wc * fn * Zprime
        Fprime = Uptdoubleprime - Wc * fn * (Z * math.pow(Zprime, 2.0) + Zdoubleprime)
        if Fprime == 0:
            Fprime = 0.000000000001 #default value by Talebpour

        Astar = Astar - (F / Fprime)

    X = Astar
    if X >= 0:
        Uptprime = Gamma1 * math.pow(X, Gamma1 - 1)
        Uptdoubleprime = Gamma1 * (Gamma1 - 1) * math.pow(X, Gamma1 - 2)
    else:
        Uptprime = Wm * Gamma2 * pow(-X, Gamma2 - 1)
        Uptdoubleprime = -Wm * Gamma2 * (Gamma2 - 1) * pow(-X, Gamma2 - 2)
    Z = (veh_delta_v + (0.5 * Astar * Tau) - (Seff / Tau)) / (Alpha * veh_delta_v)
    fn = norm.cdf(Z)
    F = Uptprime - Wc * fn * Zprime
    Fprime = Uptdoubleprime - Wc * fn * (Z * math.pow(Zprime, 2.0) + Zdoubleprime)
    if Fprime == 0:
        Fprime = 0.000000000001

    Var = -1.0 / (Beta * Fprime)

    Random_Wiener = np.random.rand()
    Yt = math.exp(-1 * 0.1 / Tau) + math.sqrt(24.0 * 0.1 / Tau) * Random_Wiener #default value by Talebpour
    accl_cf = Astar + Var * Yt
    accl_ff = accl_max * (1 - (v_a / v_desired))

    accl_ = np.minimum(accl_cf, accl_ff)

    if accl_ > 3: #default value by Talebpour
        accl_ = 3
    elif accl_ < -8: #default value by Talebpour
        accl_ = -8

    return accl_  # return acceleration
#, fn, Wc * fn are potentially other things to be returned

In [ ]:
# redefine it. Given follower id and leader id it should give me a value
def IDM(veh_id, leader_id):
    x_a = traci.vehicle.getLanePosition(veh_id)
    position=traci.vehicle.getPosition(veh_id)
    v_a=traci.vehicle.getSpeed(veh_id)
    road= traci.vehicle.getRoadID(veh_id)
    lane= traci.vehicle.getLaneIndex(veh_id)
    #print(IDMs_by_id)
    idm_params=IDMs_by_id[veh_id]
    T, a, b, v0, s0, delta = idm_params # include error term as well potentially

    if leader_id!=None:
        leader_len=lens_by_id[leader_id]
        veh_len=lens_by_id[veh_id]
        v_lead=traci.vehicle.getSpeed(leader_id)
        x_lead=traci.vehicle.getLanePosition(leader_id)
        s_star=s0+v_a*T + v_a*(v_a-v_lead)/(2*np.sqrt(a*b))
        acc=a*(1-(v_a/v0)**delta-(s_star/(x_lead-x_a-0.5*(leader_len+veh_len)))**2)
    else:
        # check the downstream of the downstream
        acc=a*(1-(v_a/v0)**delta)

    return acc


In [62]:
# we are not visualizing

#@ray.remote # need this if it is run remotely. Fine otherwise
def run_simulation(inputs):
    trial=inputs[0]
    PTs_by_id=inputs[1]
    IDMs_by_id=inputs[2]
    types_by_id=inputs[3]
    lens_by_id=inputs[4]
    trips_data=inputs[5]
    is_generated={}
    for idx in lens_by_id:
       is_generated[idx]=False 

    def IDM(veh_id, leader_id, leader_lane, adjust_length, x_a, v_a, road, lane, x_lead):
        #x_a = traci.vehicle.getLanePosition(veh_id)
        #v_a=traci.vehicle.getSpeed(veh_id)
        #road= traci.vehicle.getRoadID(veh_id)
        #lane= traci.vehicle.getLaneIndex(veh_id)
        
        idm_params=IDMs_by_id[veh_id]
        T, a, b, v0, s0, delta = idm_params # include error term as well potentially
    
        if leader_id==None:
            acc=a*(1-(v_a/v0)**delta)
        elif lane==leader_lane: # same lane segment
            leader_len=lens_by_id[leader_id]
            veh_len=lens_by_id[veh_id]
            v_lead=traci.vehicle.getSpeed(leader_id)
            x_lead=traci.vehicle.getLanePosition(leader_id)
            s_star=s0+v_a*T + v_a*(v_a-v_lead)/(2*np.sqrt(a*b))
            acc=a*(1-(v_a/v0)**delta-(s_star/(x_lead-x_a-0.5*(leader_len+veh_len)))**2)
        else: # different lane, use absolute distance
            leader_len=lens_by_id[leader_id]
            veh_len=lens_by_id[veh_id]
            v_lead=traci.vehicle.getSpeed(leader_id)
            #x_lead=traci.vehicle.getLanePosition(leader_id)
            s_star=s0+v_a*T + v_a*(v_a-v_lead)/(2*np.sqrt(a*b))
            acc=a*(1-(v_a/v0)**delta-(s_star/(x_lead+adjust_length-0.5*(leader_len+veh_len)))**2)
            
    
        return acc
    
    
    sumo_binary = checkBinary('sumo-gui')
    sim_time=max_t
    config_file = "configs/"+site+"/"+site+"_trial"+str(trial)+".sumocfg"

    all_vehs={"time":[], "veh_id":[], "type":[], "length":[],  "x":[], "y":[], "v":[], "road":[], "lane":[], "lane_pos":[] } # characteristics of vehicles of each step size appended into pd dataframe
    #all_edges_list={}
    
    print("sim_t", sim_time)
    traci.start([sumo_binary, "-c", config_file], verbose=False, label=str(trial))  # Starts the simulation
    # now set cf paramters
    #if cf_model=="IDM":
    #####    # set the cf params

    step = 0
    #veh_LC_state={} # the connectors are hard to be extracted, here I use
    lane_lens={lane: traci.lane.getLength(lane) for lane in traci.lane.getIDList()}
    
    while step < int(sim_time/step_size):  # Run the simulation for 3600 seconds (1 hour), which is 36000 steps
        if step*step_size%100==0:
            print(step*step_size)
        #'''
        # vehicle id
        veh_ids=traci.vehicle.getIDList()
        #print(veh_ids)
        
        #print(veh_ids)
        new_speeds={} # make sure there is no sequence effect
        new_lanes={} # the new lane choice of vehicles
        veh_ids_by_lane={}
        
        for veh_id in veh_ids:
            
            # just for the sake of appending
            ## the below is only for I395
            #veh_route=(traci.vehicle.getRoute(veh_id))
            #print(veh_route)
            
            #traci.vehicle.setParameter(veh_id, "noEmergencyBrake", "true")
            
            #traci.vehicle.setAllowEmergencyBrake(veh_id, False)
            if is_generated[veh_id]==False: # only set when the vehicles are first step generation
                traci.vehicle.setParameter(veh_id, "lcTimeToLat", "2.0")
                traci.vehicle.setLength(veh_id, lens_by_id[veh_id]) 
                is_generated[veh_id]=True
            
            x_a = traci.vehicle.getLanePosition(veh_id)
            position=traci.vehicle.getPosition(veh_id)
            lane_pos=traci.vehicle.getLanePosition(veh_id)
            v_a=traci.vehicle.getSpeed(veh_id)
            road= traci.vehicle.getRoadID(veh_id)
            lane= traci.vehicle.getLaneIndex(veh_id)
            #print(lane)
            #all_vehs[step][veh_id]={"x": position[0], "y": position[1], "road": road, "lane":}
            #all_vehs={"step":[], "id":[],  "x":[], "y":[], "v":[], "road":[], "lane":[], "lane_pos":[] }
            all_vehs["time"].append(step*step_size)
            all_vehs["veh_id"].append(veh_id)
            all_vehs["x"].append(position[0])
            all_vehs["y"].append(position[1])
            all_vehs["v"].append(v_a)
            all_vehs["road"].append(road)
            all_vehs["lane"].append((road)+"_"+str(lane))
            all_vehs["lane_pos"].append(lane_pos)
            all_vehs["type"].append(types_by_id[veh_id])
            all_vehs["length"].append(lens_by_id[veh_id])
            
            #[position, road, lane, traci.vehicle.getSpeed(veh_id), lane_pos]
            leader_id=traci.vehicle.getLeader(veh_id) # need to be obtained here
            next_lane=(road)+"_"+str(lane) # initialize as the current lane
            #print(leader_id, next_lanes_dict[next_lane])
            # the adjustment length
            adjust_length= lane_lens[next_lane] - traci.vehicle.getLanePosition(veh_id) #traci.lane.getLength(next_lane)-traci.vehicle.getLanePosition(veh_id) 
            # adjust_length initialized as the remaining distance of the current lane
            # this value plus the vehicle position in the current lane minus the vehicle length should be how much the gap size is
            forward_count=0
            most_upstream_dist=None
            #'''
            while leader_id==None and next_lanes_dict[next_lane]!=None and forward_count<3 and adjust_length<500 :
                forward_count=forward_count+1
                
                next_lane=next_lanes_dict[next_lane]
                if not next_lane in veh_ids_by_lane:
                    next_ids=traci.lane.getLastStepVehicleIDs(next_lane)
                    veh_ids_by_lane[next_lane]=next_ids
                else:
                    next_ids=veh_ids_by_lane[next_lane]
                    
                #print(veh_id, "next", next_ids)
                if len(next_ids)==0: # the next lane has 0 vehicles
                    adjust_length=adjust_length+lane_lens[next_lane]#traci.lane.getLength(next_lane) # the lane is skipped, so add up the length of it
                    continue
                
                most_upstream_id = min(next_ids, key=lambda vid: traci.vehicle.getLanePosition(vid))
                most_upstream_dist = traci.vehicle.getLanePosition(most_upstream_id)
                leader_id=[most_upstream_id]
                
            #'''
            leader_lane=next_lane       

            if leader_id is not None:
                leader_id=leader_id[0]
                leader_len=lens_by_id[leader_id]

            # customize car follow model
            if cf_model=="IDM":
                #print(veh_id, "next_id", leader_id)
                #print(veh_id, leader_id, leader_lane, adjust_length, x_a, v_a, road, (road)+"_"+str(lane), most_upstream_dist)
                acc=IDM(veh_id, leader_id, leader_lane, adjust_length, x_a, v_a, road, (road)+"_"+str(lane), most_upstream_dist)
                new_speeds[veh_id]=v_a+acc*step_size#max(v_a+acc*step_size, 0) # changed here

            if cf_model=="PT" or cf_model=="Prospect Theory":
                # implement prospect theory here
                acc=PT(veh_id, leader_id)
                new_speeds[veh_id]=max(v_a+acc*step_size, 0)
            
        for veh_id in veh_ids:
            # does not matter which
            #traci.vehicle.setSpeedMode(veh_id, 0)96)
            #traci.vehicle.setLaneChangeMode(veh_id, 0b001000000000) #setLaneChangeMode(veh_id, 0)
            traci.vehicle.setSpeed(veh_id, new_speeds[veh_id])
            # also set lane
            
            #road = traci.vehicle.getRoadID(veh_id)
        #'''  
        traci.simulationStep()  # Advance one simulation step
        step += 1
    
    traci.close()  # Close the simulation

    return all_vehs

In [63]:
# we are not visualizing

#@ray.remote # need this if it is run remotely. Fine otherwise
def run_simulation_default(inputs):
    trial=inputs[0]
    PTs_by_id=inputs[1]
    IDMs_by_id=inputs[2]
    types_by_id=inputs[3]
    lens_by_id=inputs[4]
    trips_data=inputs[5]

    
    
    sumo_binary = checkBinary('sumo-gui')
    sim_time=max_t
    config_file = "configs/"+site+"/"+site+"_trial"+str(trial)+".sumocfg"
    all_vehs={"time":[], "veh_id":[], "type
    ":[], "length":[],  "x":[], "y":[], "v":[], "road":[], "lane":[], "lane_pos":[] } # characteristics of vehicles of each step size appended into pd dataframe
    all_edges_list={}
    
    print("sim_t", sim_time)
    traci.start([sumo_binary, "-c", config_file], verbose=False, label=str(trial))  # Starts the simulation
    
    print()
    step = 0
    veh_LC_state={} # the connectors are hard to be extracted, here I use

    
    while step < int(sim_time/step_size):  # Run the simulation for 3600 seconds (1 hour), which is 36000 steps
        if step*step_size%100==0:
            print(step*step_size)
        #'''
        veh_ids=traci.vehicle.getIDList()
        
        #print(veh_ids)
        new_speeds={} # make sure there is no sequence effect
        new_lanes={} # the new lane choice of vehicles
        for veh_id in veh_ids:
            # just for the sake of appending
            #print(traci.vehicle.getRoute(veh_id))
            traci.vehicle.setLength(veh_id, lens_by_id[veh_id])
            if types_by_id[veh_id]==1:
                traci.vehicle.setVehicleClass(veh_id, "passenger")
            if types_by_id[veh_id]==1:
                traci.vehicle.setColor(veh_id, (0,255,0))
                traci.vehicle.setVehicleClass(veh_id, "passenger")
            if types_by_id[veh_id]==2:

                traci.vehicle.setVehicleClass(veh_id, "truck")
                traci.vehicle.setParameter(veh_id, "guiShape", "truck")

            
            x_a = traci.vehicle.getLanePosition(veh_id)
            position=traci.vehicle.getPosition(veh_id)
            lane_pos=traci.vehicle.getLanePosition(veh_id)
            v_a=traci.vehicle.getSpeed(veh_id)
            road= traci.vehicle.getRoadID(veh_id)
            if not road in all_edges_list:
                all_edges_list[road]={}
                all_edges_list[road]["x"]=[]
                all_edges_list[road]["y"]=[]
            all_edges_list[road]["x"].append(position[0])
            all_edges_list[road]["y"].append(position[1])

            lane= traci.vehicle.getLaneIndex(veh_id)
            #print(lane)
            #all_vehs[step][veh_id]={"x": position[0], "y": position[1], "road": road, "lane":}
            #all_vehs={"step":[], "id":[],  "x":[], "y":[], "v":[], "road":[], "lane":[], "lane_pos":[] }
            all_vehs["time"].append(step*step_size)
            all_vehs["veh_id"].append(veh_id)
            all_vehs["x"].append(position[0])
            all_vehs["y"].append(position[1])
            all_vehs["v"].append(v_a)
            all_vehs["road"].append(road)
            all_vehs["lane"].append((road)+"_lane_"+str(lane))
            all_vehs["lane_pos"].append(lane_pos)
            all_vehs["type"].append(types_by_id[veh_id])
            all_vehs["length"].append(lens_by_id[veh_id])
            
        traci.simulationStep()  # Advance one simulation step
        step += 1
    
    traci.close()  # Close the simulation

    return all_vehs

SyntaxError: unterminated string literal (detected at line 17) (3195535967.py, line 17)

In [64]:
# pick the site
site_idx=3
sites={1:"I90_94", 2:"I294", 3:"I395" }
site=sites[site_idx]


min_hw=0.1 # the minimum headway between vehicles during vehicle generation. Not sure if this is needed
# the min and max sim time
min_t=0
max_t=1200 # change this for a different simulation time

# simulation step size
step_size=0.1

# Model selection
cf_model_choice=1
cf_models = {1:"IDM", 2:"PT"} # car following model

# number_of trails
num_trials = 1

warm_up_t=200

In [65]:
#params=[av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice]# 


all_scenarios=[[0.1, 0.1, 2000, 50, 100, 100, 1],
              [0.2, 0.1, 2000, 50, 100, 100, 1],
              [0.5, 0.1, 2000, 50, 100, 100, 1],
              [0.2, 0.1, 2000, 50, 100, 100, 1],
              [0.2, 0.2, 2000, 50, 100, 100, 1],
              [0.2, 0.5, 2000, 50, 100, 100, 1],
               
              [0.1, 0.1, 1000, 25, 50, 50, 1],
              [0.5, 0.1, 1000, 25, 50, 50, 1],
              [0.2, 0.1, 1000, 25, 50, 50, 1],
              [0.2, 0.5, 1000, 25, 50, 50, 1]]


# the ones to be tested
all_scenarios=[[0.1, 0.1, 4000, 200, 400, 400, 1],
              [0.2, 0.1, 4000, 200, 400, 400, 1],
              [0.5, 0.1, 4000, 200, 400, 400, 1],
              [0.8, 0.1, 4000, 200, 400, 400, 1],
              
              [0.2, 0.2, 4000, 200, 400, 400, 1],
              [0.2, 0.5, 4000, 200, 400, 400, 1],
               
              [0.1, 0.1, 2000, 100, 200, 200, 1],
              [0.5, 0.1, 2000, 100, 200, 200, 1],
              [0.2, 0.1, 2000, 100, 200, 200, 1],
              [0.2, 0.5, 2000, 100, 200, 200, 1]]

all_scenarios=[[0.3, 0.2, 6000, 500, 500, 500, 1]]

In [66]:
traci.close()

In [67]:
# one at a time
# make sure no ray module before the function
#'''
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    inputs_all_trial=[]
    od_demand, all_edges, in_edges, out_edges, first_exit, first_exit_start_lane, next_lanes_dict=initialize()
    for trial in range(0,1): # number of trials
        print("trial", trial)
        
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data= generate_vehicles(trial)
        inputs_all_trial.append([trial, PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data])
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = None, None, None, None, None
        
    vehs_all_trials=[run_simulation_default(inputs) for inputs in inputs_all_trial]
    
    for trial in range(0,1):
        all_vehs=pd.DataFrame(vehs_all_trials[trial])
        all_vehs.to_csv("trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")
#'''

[0.3, 0.2, 6000, 500, 500, 500, 1]
From main_in to off1:  500 veh/hr
From main_in to main_out:  6000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to main_out:  500 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
trial 0
Trips saved to configs/I395/I395_trial0.trips.xml
sim_t 1200

0.0
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0


In [33]:
# parallel
n_job=1
ray.init(num_cpus=1)
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    inputs_all_trial=[]
    od_demand, all_edges, in_edges, out_edges, first_exit, first_exit_start_lane, next_lanes_dict=initialize()
    for trial in range(0,num_trials): # number of trials
        print("trial", trial)
        
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data= generate_vehicles(trial)
        inputs_all_trial.append([trial, PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data])
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = None, None, None, None, None
        
    vehs_all_trials=ray.get([run_simulation.remote(inputs) for inputs in inputs_all_trial])
    
    for trial in range(0,num_trials):
        all_vehs=pd.DataFrame(vehs_all_trials[trial])
        all_vehs.to_csv("sim_data/trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")

ray.shutdown()

2025-03-21 15:31:34,247	INFO worker.py:1841 -- Started a local Ray instance.


[0.1, 0.1, 4000, 200, 400, 400, 1]
From main_in to off1:  400 veh/hr
From main_in to off2:  400 veh/hr
From main_in to off3:  400 veh/hr
From main_in to off4:  400 veh/hr
From main_in to main_I90_out:  4000 veh/hr
From main_in to main_I94_out:  4000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to off2:  200 veh/hr
From on1 to off3:  200 veh/hr
From on1 to off4:  200 veh/hr
From on1 to main_I90_out:  400 veh/hr
From on1 to main_I94_out:  400 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
trial 0
Trips saved to configs/I90_94/I90_94_trial0.trips.xml
(run_simulation pid=4660) sim_t 2500
(run_simulation pid=4660) 0.0
(run_simulation pid=4660) 100.0
(run_simulation pid=4660) 200.0
(run_simulation pid=4660) 300.0
(run_simulation pid=4660) 400.0
(run_simulation pid=4660) 500.0
(run_simulation pid=4660) 600.0
(run_simul

RayTaskError(FatalTraCIError): [36mray::run_simulation()[39m (pid=4660, ip=127.0.0.1)
  File "python\\ray\\_raylet.pyx", line 1883, in ray._raylet.execute_task
  File "C:\Users\nachu\AppData\Local\Temp\ipykernel_8360\577545243.py", line 53, in run_simulation
  File "C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py", line 147, in start
    result = init(sumoPort, numRetries, "localhost", label, sumoProcess, doSwitch, traceFile, traceGetters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files (x86)\Eclipse\Sumo\tools\traci\main.py", line 119, in init
    return con.getVersion()
           ^^^^^^^^^^^^^^^^
  File "C:\Program Files (x86)\Eclipse\Sumo\tools\traci\connection.py", line 382, in getVersion
    result = self._sendCmd(command, None, None)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files (x86)\Eclipse\Sumo\tools\traci\connection.py", line 232, in _sendCmd
    return s

In [32]:
ray.shutdown()
#traci.close()

In [13]:
next_lanes_dict['124471132_1']

':124471132-AddedOffRampNode_0_2'

In [16]:
# parallel
n_job=5
ray.init(num_cpus=5)
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    inputs_all_trial=[]
    od_demand, all_edges, in_edges, out_edges=initialize()
    for trial in range(0,5): # number of trials
        print("trial", trial)
        
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = generate_vehicles(trial)
        inputs_all_trial.append([trial, PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data])
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = None, None, None, None, None
        
    vehs_all_trials=ray.get([run_simulation.remote(inputs) for inputs in inputs_all_trial])
    
    for trial in range(0,5):
        all_vehs=pd.DataFrame(vehs_all_trials[trial])
        all_vehs.to_csv("sim_data/trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")

2025-03-13 14:57:33,536	INFO worker.py:1841 -- Started a local Ray instance.


[0.1, 0.1, 2000, 50, 100, 100, 1]
From main_in to off1:  100 veh/hr
From main_in to main_out:  2000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to main_out:  100 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
trial 0
Trips saved to configs/I395_trial0.trips.xml
trial 1
Trips saved to configs/I395_trial1.trips.xml
trial 2
Trips saved to configs/I395_trial2.trips.xml
trial 3
Trips saved to configs/I395_trial3.trips.xml
trial 4


KeyboardInterrupt: 

In [28]:
all_scenarios=[          
               [0.1, 0.1, 2000, 50, 100, 100, 1],
               [0.2, 0.1, 2000, 50, 100, 100, 1],
               [0.5, 0.1, 2000, 50, 100, 100, 1],
               [0.8, 0.1, 2000, 50, 100, 100, 1]]

In [29]:
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    
    for trial in range(0,num_trials): # number of trials
        print("trial", trial)
        od_demand, all_edges, in_edges, out_edges=initialize()
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = generate_vehicles()
        all_vehs=run_simulation()

        all_vehs=pd.DataFrame(all_vehs)
        all_vehs.to_csv("sim_data/trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")
    
    

[0.1, 0.1, 2000, 50, 100, 100, 1]
trial 0
From main_in to off1:  100 veh/hr
From main_in to main_out:  2000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to main_out:  100 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
Trips saved to I395.trips.xml
sim_t 1900
0.0
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
trial 1
From main_in to off1:  100 veh/hr
From main_in to main_out:  2000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to main_out:  100 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
Trips saved to I395.trips.xml
sim_t 1900
0.0
100.0
200.0
300.0
400.0
500.0
600.0
7

In [18]:
site_idx=1
sites={1:"I90_94", 2:"I294", 3:"I395" }
site=sites[site_idx]

In [19]:
for scenarios in all_scenarios:
    print(scenarios)
    av_rate, lv_rate, flow_main, flow_ramp_to_ramp, flow_ramp_to_main, flow_main_to_ramp, cf_model_choice = scenarios
    sv_rate=1-av_rate-lv_rate
    cf_model= cf_models[cf_model_choice]
    
    for trial in range(0,num_trials): # number of trials
        print("trial", trial)
        od_demand, all_edges, in_edges, out_edges=initialize()
        PTs_by_id,  IDMs_by_id, types_by_id, lens_by_id, trips_data = generate_vehicles()
        all_vehs=run_simulation()

        all_vehs=pd.DataFrame(all_vehs)
        all_vehs.to_csv("sim_data/trajs_"+site+"_"+str(scenarios)+"_trial"+str(trial)+".csv")

[0.8, 0.1, 1000, 50, 100, 100, 1]
trial 0
From main_in to off1:  100 veh/hr
From main_in to off2:  100 veh/hr
From main_in to off3:  100 veh/hr
From main_in to off4:  100 veh/hr
From main_in to main_I90_out:  1000 veh/hr
From main_in to main_I94_out:  1000 veh/hr
From on1 to off1:  0 veh/hr
From on1 to off2:  50 veh/hr
From on1 to off3:  50 veh/hr
From on1 to off4:  50 veh/hr
From on1 to main_I90_out:  100 veh/hr
From on1 to main_I94_out:  100 veh/hr
Root tag: net, Root attributes: {'version': '1.20', 'junctionCornerDetail': '5', 'limitTurnSpeed': '5.50', '{http://www.w3.org/2001/XMLSchema-instance}noNamespaceSchemaLocation': 'http://sumo.dlr.de/xsd/net_file.xsd'}
Trips saved to I90_94.trips.xml
sim_t 1900
0.0
100.0
200.0
300.0
400.0
500.0
600.0
700.0
800.0
900.0
1000.0
1100.0
1200.0
1300.0
1400.0
1500.0
1600.0
1700.0
1800.0
trial 1
From main_in to off1:  100 veh/hr
From main_in to off2:  100 veh/hr
From main_in to off3:  100 veh/hr
From main_in to off4:  100 veh/hr
From main_in to mai

MemoryError: 

In [20]:
len(all_edges_list[road]["x"])

NameError: name 'all_edges_list' is not defined

In [ ]:
for trial in range(0,num_trials):
    all_vehs=pd.DataFrame(all_datas[trial])

    all_lanes=all_vehs["lane"].unique()
    all_roads=all_vehs["road"].unique()
    all_spds={}
    all_dens={}
    all_flows={}
    spds=[]
    dens=[]
    flows=[]
    t_wd=20
    x_wd=50
    A=t_wd*x_wd
    warm_up_t=50
    for road in all_roads:
        road_data=all_vehs[all_vehs["road"]==road]
        lanes=road_data["lane"].unique()
        #print(lanes, len(lanes))
        if len(lanes)==1:
            # if the lane belongs to on-ramp or off-ramp, we do not analyze
            continue
        for lane in lanes:
            all_spds[lane]={}
            all_dens[lane]={}
            all_flows[lane]={}
            #print(lane)
            lane_data=road_data[road_data["lane"]==lane]
            #min_x=min(lane_data["lane_pos"])
            max_x=max(lane_data["lane_pos"])
            num_x_wds=int(max_x//x_wd)
            for t in range(warm_up_t,max_t, t_wd):
                all_spds[lane][t]={}
                all_dens[lane][t]={}
                all_flows[lane][t]={}
                t_low=t
                t_high=t+t_wd
                data_t=lane_data[(lane_data["time"]>=t_low) &  (lane_data["time"]<t_high)]
                for x_i in range(0, num_x_wds):
                    x_low=x_i*x_wd
                    x_high=x_low+x_wd
                    slc=data_t[(data_t["lane_pos"]>=x_low) & (data_t["lane_pos"]<x_high)]
                    tot_t=len(slc)*step_size
                    tot_x=sum(slc["v"])*step_size
                    if len(slc)!=0:
                        spd=np.average(slc["v"].values)
                        density=tot_t/A
                        flow=tot_x/A
                    else:
                        spd=np.nan
                        density=np.nan
                        flow=np.nan
                    all_spds[lane][t][x_low]=spd
                    all_dens[lane][t][x_low]=density*1000
                    all_flows[lane][t][x_low]=flow*3600
                    spds.append(spd)
                    dens.append(density*1000)
                    flows.append(flow*3600)
    
    plt.scatter(dens, flows, s=1)
    plt.xlabel("Density (Veh/km)")
    plt.ylabel("Flow (Veh/hr)")
    plt.xlim(0,100)
    plt.ylim(0,2000)
    plt.show()
                
                
                
                

In [ ]:
# run this to close traci run time
# traci.close()

In [ ]:
# perform analysis